In [1]:
from datetime import datetime, timedelta
from scipy.optimize import fsolve
import calendar

class CouponBond:
    def __init__(self, issue_date, maturity_date, coupon_rate, face_value=100):
        self.issue_date = datetime.strptime(issue_date, "%m/%d/%Y")
        self.maturity_date = datetime.strptime(maturity_date, "%m/%d/%Y")
        self.coupon_rate = coupon_rate
        self.face_value = face_value
        self.coupon_frequency = 2  # Semi-annual

    def _adjust_for_weekend(self, date):
        
        if date.weekday() == 5:  # Saturday
            return date + timedelta(days=2)
        elif date.weekday() == 6:  # Sunday
            return date + timedelta(days=1)
        return date

    def _days_in_february(self, year):
        
        return 29 if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)) else 28

    def _validate_date(self, year, month, day):
        
        last_day = calendar.monthrange(year, month)[1]  # Get the last valid day of the month
        if day > last_day:
            return datetime(year, month, last_day)  # Adjust to the last valid day of the month
        return datetime(year, month, day)

    def _generate_coupon_dates(self, settlement_date):
        
        if isinstance(settlement_date, str):
            settlement_date = datetime.strptime(settlement_date, "%m/%d/%Y")

        maturity_month = self.maturity_date.month
        maturity_day = self.maturity_date.day

        if maturity_month > 6:
            int_date1_month = maturity_month - 6
            int_date1_day = maturity_day
            int_date2_month = maturity_month
            int_date2_day = maturity_day
        else:
            int_date1_month = maturity_month
            int_date1_day = maturity_day
            int_date2_month = maturity_month + 6
            int_date2_day = maturity_day

        coupon_dates = []
        year = self.issue_date.year

        last_coupon_date_before_settlement = None

        while True:
            # Validate and adjust dates for invalid month-end days
            int_date1 = self._validate_date(year, int_date1_month, int_date1_day)
            int_date2 = self._validate_date(year, int_date2_month, int_date2_day)

            int_date1 = self._adjust_for_weekend(int_date1)
            int_date2 = self._adjust_for_weekend(int_date2)

            if int_date1 < settlement_date:
                last_coupon_date_before_settlement = int_date1
            elif int_date1 > settlement_date and int_date1 <= self.maturity_date:
                coupon_dates.append(int_date1)

            if int_date2 < settlement_date:
                last_coupon_date_before_settlement = int_date2
            elif int_date2 > settlement_date and int_date2 <= self.maturity_date:
                coupon_dates.append(int_date2)

            if year >= self.maturity_date.year:
                break

            year += 1

        if last_coupon_date_before_settlement:
            coupon_dates.insert(0, last_coupon_date_before_settlement)

        if self.maturity_date not in coupon_dates:
            coupon_dates.append(self._adjust_for_weekend(self.maturity_date))

        return sorted(coupon_dates)

    def future_cashflow(self, settlement_date):
        
        if isinstance(settlement_date, str):
            settlement_date = datetime.strptime(settlement_date, "%m/%d/%Y")
    
        coupon_dates = self._generate_coupon_dates(settlement_date)
        cashflows = []
    
        # Adjust the maturity date for weekends
        adjusted_maturity_date = self._adjust_for_weekend(self.maturity_date)
    
        # Filter future coupon dates after the settlement date
        future_coupon_dates = [date for date in coupon_dates if date > settlement_date]
    
        for i, date in enumerate(future_coupon_dates):
            # Calculate the coupon payment amount
            cashflow = (self.coupon_rate * self.face_value) / self.coupon_frequency
    
            # Ensure only the date part is used for comparison with the adjusted maturity date
            if date.date() == adjusted_maturity_date.date():  # Convert both to date for accurate comparison
                cashflow += self.face_value  # Add principal repayment
    
            # Calculate earned days for the first coupon payment
            if i == 0:
                earned_days = (date - settlement_date).days  # Days between settlement date and the first coupon date
            else:
                earned_days = (date - future_coupon_dates[i - 1]).days  # Days between consecutive coupon dates
    
            # Calculate coupon period days between last coupon date and current coupon date
            if i == 0:
                last_coupon_date = max([d for d in coupon_dates if d <= settlement_date], default=self.issue_date)
                coupon_period_days = (date - last_coupon_date).days
            else:
                coupon_period_days = (date - future_coupon_dates[i - 1]).days
    
            cashflows.append({
                "date": date,
                "cashflow": cashflow,
                "earned_days": earned_days,
                "coupon_period_days": coupon_period_days,
            })
    
        return cashflows




    def accrued_interest(self, settlement_date):
        
        if isinstance(settlement_date, str):
            settlement_date = datetime.strptime(settlement_date, "%m/%d/%Y")
    
        coupon_dates = self._generate_coupon_dates(settlement_date)
        last_coupon_date = max([d for d in coupon_dates if d <= settlement_date], default=None)
    
        if not last_coupon_date:
            raise ValueError("No valid last coupon date found before the settlement date.")
    
        next_coupon_date = min([d for d in coupon_dates if d > last_coupon_date], default=None)
    
        if not next_coupon_date:
            raise ValueError("No valid next coupon date found after the settlement date.")
    
        days_in_period = (next_coupon_date - last_coupon_date).days
        days_since_last_coupon = (settlement_date - last_coupon_date).days
    
        accrued_interest = (self.coupon_rate * self.face_value / self.coupon_frequency) * (days_since_last_coupon / days_in_period)
    
        return accrued_interest, days_since_last_coupon, days_in_period
        

    def bey_from_clean_price(self, clean_price, settlement_date):
        
        if isinstance(settlement_date, str):
            settlement_date = datetime.strptime(settlement_date, "%m/%d/%Y")
    
        # Generate the future cashflows
        cashflows = self.future_cashflow(settlement_date)
    
        # Calculate the discount powers
        discount_powers = []
        accumulated_power = 0
        for i, cashflow in enumerate(cashflows):
            if i == 0:
               
                power = cashflow['earned_days'] / cashflow['coupon_period_days']
                accumulated_power = power  # Initialize accumulated power with the first power
            else:
                
                power = cashflow['earned_days'] / cashflow['coupon_period_days']
                accumulated_power += power
            discount_powers.append(accumulated_power)
    
       
        accrued_interest, _, _ = self.accrued_interest(settlement_date)
        dirty_price = clean_price + accrued_interest
    
        # Print the discount powers for examination
        #print("Discount Powers for Each Cash Flow:")
        #for i, power in enumerate(discount_powers):
            #print(f"Cashflow {i + 1}: Power = {power:.4f}") 
  
        # solving BEY
        def yield_function(yield_rate):
            total_pv = 0
            for i, cashflow in enumerate(cashflows):
                # Apply the discount using the calculated power for each cash flow
                total_pv += cashflow['cashflow'] / ((1 + yield_rate / self.coupon_frequency) ** discount_powers[i])
            return total_pv - dirty_price
    
        #  fsolve to find the yield rate
        initial_guess = 0.05  # Initial guess for the BEY
        result = fsolve(yield_function, initial_guess)

        return result[0]
       

    def dirty_price_from_bey(self, bey, settlement_date):
        
        if isinstance(settlement_date, str):
            settlement_date = datetime.strptime(settlement_date, "%m/%d/%Y")
    
        # Generate the future cashflows
        cashflows = []
        cashflows = self.future_cashflow(settlement_date)
    
        # Calculate the discount powers
        discount_powers = []
        accumulated_power = 0
        for i, cashflow in enumerate(cashflows):
            if i == 0:
                
                power = cashflow['earned_days'] / cashflow['coupon_period_days']
                accumulated_power = power  # Initialize accumulated power with the first power
            else:
                
                power = cashflow['earned_days'] / cashflow['coupon_period_days']
                accumulated_power += power
            discount_powers.append(accumulated_power)
    
        
        total_pv = 0
        for i, cashflow in enumerate(cashflows):
            pv = cashflow['cashflow'] / ((1 + bey / self.coupon_frequency) ** discount_powers[i])
            total_pv += pv
            # Print details for examination
            #print(f"Cashflow {i + 1}: Date = {cashflow['date']}, Cashflow = {cashflow['cashflow']:.2f}, "
                  #f"Discount Power = {discount_powers[i]:.4f}, PV = {pv:.10f}")
    
        # The total present value represents the dirty price
        #print(f"Total Present Value (Dirty Price): {total_pv:.10f}")
        return total_pv

    def modified_duration(self, bey, settlement_date):
        """Calculate the modified duration of the bond."""
        if isinstance(settlement_date, str):
            settlement_date = datetime.strptime(settlement_date, "%m/%d/%Y")

     
        P0 = self.dirty_price_from_bey(bey, settlement_date)

        P_plus = self.dirty_price_from_bey(bey + 0.0001, settlement_date)

        P_minus = self.dirty_price_from_bey(bey - 0.0001, settlement_date)

       
        delta_y = 0.0001  # Change in yield (1 basis point)
        mod_duration = (P_minus - P_plus) / (2 * P0 * delta_y)

        return mod_duration


In [2]:
# Example usage 1
bond = CouponBond("09/15/2023", "09/15/2026", 0.04625)  # Coupon rate in decimal format
settlement_date = "09/02/2024"

# Calculate future cash flows
bond.future_cashflow(settlement_date)

[{'date': datetime.datetime(2024, 9, 16, 0, 0),
  'cashflow': 2.3125,
  'earned_days': 14,
  'coupon_period_days': 185},
 {'date': datetime.datetime(2025, 3, 17, 0, 0),
  'cashflow': 2.3125,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2025, 9, 15, 0, 0),
  'cashflow': 2.3125,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2026, 3, 16, 0, 0),
  'cashflow': 2.3125,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2026, 9, 15, 0, 0),
  'cashflow': 102.3125,
  'earned_days': 183,
  'coupon_period_days': 183}]

In [4]:
bond.accrued_interest(settlement_date)

(2.1375, 171, 185)

In [3]:
clean_price = 101.34375
bond.bey_from_clean_price(clean_price, settlement_date)

0.03931561756956373

In [4]:
bond.dirty_price_from_bey(0.03931561756956373, "09/02/2024")

103.48125000000003

In [5]:
bond.modified_duration(0.03931561756956373, "09/02/2024")

1.8912201833691387

In [7]:
#Example Usage 2
a= CouponBond("03/01/2021", "02/29/2028", 0.01125)  
settlement_date = "09/02/2024"

In [8]:
a.future_cashflow(settlement_date)

[{'date': datetime.datetime(2025, 2, 28, 0, 0),
  'cashflow': 0.5625,
  'earned_days': 179,
  'coupon_period_days': 183},
 {'date': datetime.datetime(2025, 8, 29, 0, 0),
  'cashflow': 0.5625,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2026, 3, 2, 0, 0),
  'cashflow': 0.5625,
  'earned_days': 185,
  'coupon_period_days': 185},
 {'date': datetime.datetime(2026, 8, 31, 0, 0),
  'cashflow': 0.5625,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2027, 3, 1, 0, 0),
  'cashflow': 0.5625,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2027, 8, 30, 0, 0),
  'cashflow': 0.5625,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2028, 2, 29, 0, 0),
  'cashflow': 100.5625,
  'earned_days': 183,
  'coupon_period_days': 183}]

In [9]:
a.accrued_interest(settlement_date)

(0.012295081967213115, 4, 183)

In [10]:
clean_price = 91.375
a.bey_from_clean_price(clean_price, settlement_date)

0.03787186265193255

In [11]:
a.dirty_price_from_bey(0.03787186265193255, "09/02/2024")

91.38729508196717

In [12]:
a.modified_duration(0.037871845, "09/02/2024")

3.3638490731177177

In [11]:
b= CouponBond("02/15/2001", "02/15/2031", 0.05375)
settlement_date = "09/02/2024"

In [12]:
b.future_cashflow(settlement_date)

[{'date': datetime.datetime(2025, 2, 17, 0, 0),
  'cashflow': 2.6875,
  'earned_days': 168,
  'coupon_period_days': 186},
 {'date': datetime.datetime(2025, 8, 15, 0, 0),
  'cashflow': 2.6875,
  'earned_days': 179,
  'coupon_period_days': 179},
 {'date': datetime.datetime(2026, 2, 16, 0, 0),
  'cashflow': 2.6875,
  'earned_days': 185,
  'coupon_period_days': 185},
 {'date': datetime.datetime(2026, 8, 17, 0, 0),
  'cashflow': 2.6875,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2027, 2, 15, 0, 0),
  'cashflow': 2.6875,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2027, 8, 16, 0, 0),
  'cashflow': 2.6875,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datetime(2028, 2, 15, 0, 0),
  'cashflow': 2.6875,
  'earned_days': 183,
  'coupon_period_days': 183},
 {'date': datetime.datetime(2028, 8, 15, 0, 0),
  'cashflow': 2.6875,
  'earned_days': 182,
  'coupon_period_days': 182},
 {'date': datetime.datet

In [731]:
b.accrued_interest(settlement_date)

(0.2600806451612903, 18, 186)

In [732]:
clean_price = 109.15625
b.bey_from_clean_price(clean_price, settlement_date)

0.0376291732233672

In [734]:
b.dirty_price_from_bey(0.0376291732233672, "09/02/2024")

109.41633064516128

In [735]:
b.modified_duration(-0.2694862525964172, "09/02/2024")

7.114947474536079

In [ ]:
#######Part b

In [13]:
import pandas as pd
bills_notes_bonds = pd.read_csv('BillsNotesBondsOutstanding_20240831.csv')
coupon_security_prices = pd.read_csv('CouponSecurityPrices_20240831.csv')

In [14]:
bills_notes_bonds.columns = bills_notes_bonds.iloc[0]  
bills_notes_bonds = bills_notes_bonds[1:]  


bills_notes_bonds = bills_notes_bonds[['CUSIP', 'IssueDate']]
bills_notes_bonds['IssueDate'] = pd.to_datetime(bills_notes_bonds['IssueDate'], errors='coerce')


coupon_security_prices = coupon_security_prices.merge(bills_notes_bonds, on='CUSIP', how='inner')
coupon_security_prices['MaturityDate'] = pd.to_datetime(coupon_security_prices['MaturityDate'], errors='coerce')
coupon_security_prices['SettlementDate'] = pd.to_datetime(coupon_security_prices['SettlementDate'], errors='coerce')
coupon_security_prices['IssueDate'] = pd.to_datetime(coupon_security_prices['IssueDate'], errors='coerce')


coupon_security_prices['MaturityDate'] = coupon_security_prices['MaturityDate'].dt.strftime('%m/%d/%Y')
coupon_security_prices['SettlementDate'] = coupon_security_prices['SettlementDate'].dt.strftime('%m/%d/%Y')
coupon_security_prices['IssueDate'] = coupon_security_prices['IssueDate'].dt.strftime('%m/%d/%Y')
df = coupon_security_prices
# Display the merged DataFrame
df

,SettlementDate,CUSIP,CouponRate,MaturityDate,CleanPrice,IssueDate
0,09/02/2024,91282CCX7,0.00375,09/15/2024,99.81250,09/15/2021
1,09/02/2024,912828YH7,0.01500,09/30/2024,99.71875,09/30/2019
2,09/02/2024,9128282Y5,0.02125,09/30/2024,99.78125,10/02/2017
3,09/02/2024,91282CFN6,0.04250,09/30/2024,99.90625,09/30/2022
4,09/02/2024,91282CDB4,0.00625,10/15/2024,99.46875,10/15/2021
...,...,...,...,...,...,...
581,09/02/2024,912810TX6,0.04250,02/15/2054,100.62500,04/15/2024
582,09/02/2024,912810UA4,0.04625,05/15/2054,107.12500,05/15/2024
583,09/02/2024,912810UA4,0.04625,05/15/2054,107.12500,06/17/2024
584,09/02/2024,912810UA4,0.04625,05/15/2054,107.12500,07/15/2024


In [15]:

def calculate_bey_and_duration(df):
    beys = []
    mod_durations = []

    for index, row in df.iterrows():
        try:
            
            bond = CouponBond(
                issue_date=row['IssueDate'],  
                maturity_date=row['MaturityDate'],
                coupon_rate=row['CouponRate'],  
                face_value=row['FaceValue'] if 'FaceValue' in row else 100  # Default to 100 if FaceValue not provided
            )

            
            clean_price = row['CleanPrice']  
            settlement_date = row['SettlementDate']  
            bey = bond.bey_from_clean_price(clean_price, settlement_date)
            beys.append(bey)

            
            mod_duration = bond.modified_duration(bey, settlement_date)
            mod_durations.append(mod_duration)

        except Exception as e:
            beys.append(float('nan'))
            mod_durations.append(float('nan'))
            print(f"Error processing bond at index {index}: {e}")

    
    df['BEY'] = beys
    df['ModifiedDuration'] = mod_durations

    return df


coupon_security_prices = calculate_bey_and_duration(coupon_security_prices)


coupon_security_prices


,SettlementDate,CUSIP,CouponRate,MaturityDate,CleanPrice,IssueDate,BEY,ModifiedDuration
0,09/02/2024,91282CCX7,0.00375,09/15/2024,99.81250,09/15/2021,0.053973,0.036844
1,09/02/2024,912828YH7,0.01500,09/30/2024,99.71875,09/30/2019,0.051943,0.074976
2,09/02/2024,9128282Y5,0.02125,09/30/2024,99.78125,10/02/2017,0.049870,0.075052
3,09/02/2024,91282CFN6,0.04250,09/30/2024,99.90625,09/30/2022,0.054387,0.074887
4,09/02/2024,91282CDB4,0.00625,10/15/2024,99.46875,10/15/2021,0.052131,0.114502
...,...,...,...,...,...,...,...,...
581,09/02/2024,912810TX6,0.04250,02/15/2054,100.62500,04/15/2024,0.042126,16.719843
582,09/02/2024,912810UA4,0.04625,05/15/2054,107.12500,05/15/2024,0.042025,16.334763
583,09/02/2024,912810UA4,0.04625,05/15/2054,107.12500,06/17/2024,0.042025,16.334763
584,09/02/2024,912810UA4,0.04625,05/15/2054,107.12500,07/15/2024,0.042025,16.334763


In [16]:
coupon_security_original = pd.read_csv('CouponSecurityPrices_20240831.csv')
coupon_security_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SettlementDate  340 non-null    object 
 1   CUSIP           340 non-null    object 
 2   CouponRate      340 non-null    float64
 3   MaturityDate    340 non-null    object 
 4   CleanPrice      340 non-null    float64
dtypes: float64(2), object(3)
memory usage: 13.4+ KB


In [17]:
mod_df = []
for i in range(len(coupon_security_original)):
    filter_df = coupon_security_prices[(coupon_security_prices['CUSIP'] == 
                        coupon_security_original['CUSIP'][i]) & (coupon_security_prices['MaturityDate'] == coupon_security_original['MaturityDate'][i])]
    mod_df.append(filter_df.reset_index(drop=True).iloc[0])
mod_df = pd.DataFrame(mod_df).drop(columns='IssueDate').rename(columns={'BEY':'yield','ModifiedDuration':'moddur'})
mod_df

,SettlementDate,CUSIP,CouponRate,MaturityDate,CleanPrice,yield,moddur
0,09/02/2024,91282CCX7,0.00375,09/15/2024,99.81250,0.053973,0.036844
0,09/02/2024,912828YH7,0.01500,09/30/2024,99.71875,0.051943,0.074976
0,09/02/2024,9128282Y5,0.02125,09/30/2024,99.78125,0.049870,0.075052
0,09/02/2024,91282CFN6,0.04250,09/30/2024,99.90625,0.054387,0.074887
0,09/02/2024,91282CDB4,0.00625,10/15/2024,99.46875,0.052131,0.114502
...,...,...,...,...,...,...,...
0,09/02/2024,912810TT5,0.04125,08/15/2053,98.43750,0.042189,16.671970
0,09/02/2024,912810TV0,0.04750,11/15/2053,109.18750,0.042006,16.105516
0,09/02/2024,912810TX6,0.04250,02/15/2054,100.62500,0.042126,16.719843
0,09/02/2024,912810UA4,0.04625,05/15/2054,107.12500,0.042025,16.334763


In [62]:
mod_df.to_csv('CouponSecurityPYD_20240831.csv', index=False)
print("BEY and Modified Duration calculation complete. Results saved to CouponSecurityPYD_20240831.csv.")

BEY and Modified Duration calculation complete. Results saved to CouponSecurityPYD_20240831.csv.
